In [1]:
import findspark
findspark.init('/opt/cloudera/parcels/SPARK2-2.4.0.cloudera2-1.cdh5.13.3.p0.1041012/lib/spark2/')
import os, findspark
os.environ['PYSPARK_PYTHON'] = '/home/tozeng/anaconda3/bin/python'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/home/tozeng/anaconda3/bin/python'
import sys
sys.path.append('/home/eileen/nsf_data_ingestion/')
import zipfile
import zipimport
import io
import logging
logging.getLogger().setLevel(logging.INFO)
from pyspark.sql import functions
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import Row
from pyspark.sql import Window
from pyspark.sql.functions import rank, max, sum, desc
import zlib
import importlib
import os
from os import path
from shutil import copyfile
from shutil import rmtree
from subprocess import call
import pubmed_parser as pp
import pyarrow as pa
import subprocess

#sys.path.append('/home/ananth/nsf_data_ingestion/')

from nsf_data_ingestion.config import spark_config
from nsf_data_ingestion.objects import data_source_params

namenode_url = "hdfs://ist-deacuna-n1.syr.edu:8020/"

In [2]:
def create_session():
    logging.info('Creating Spark Session....')
    spark = SparkSession.builder.config("spark.executor.instances", spark_config.exec_instance).\
                                 config("spark.executor.memory", spark_config.exec_mem).\
                                 config('spark.executor.cores', spark_config.exec_cores).\
                                 config('spark.cores.max', spark_config.exec_max_cores).\
                                 config('spark.jars', '/home/eileen/nsf_data_ingestion/libraries/spark-xml_2.11-0.5.0.jar').\
                                 appName('medline_parquet_test').getOrCreate()
    spark.sparkContext.addPyFile('/home/eileen/nsf_data_ingestion/libraries/pubmed_parser-0.1.0-py3.6.egg')
    spark.sparkContext.addPyFile('/home/eileen/nsf_data_ingestion/libraries/Unidecode-1.1.1-py3.6.egg')
#         logging.info('Adding Libraries' + str(library))
#         spark.sparkContext.addPyFile(library)    # adding libraries
#         spark.sparkContext.addPyFile(library)

        
    return spark
spark = create_session()


INFO:root:Creating Spark Session....


In [3]:
hdfs = pa.hdfs.connect()
file_paths = hdfs.ls('test/test')
hdfs.ls('test/test')

['/user/eileen/test/test/pubmed21n0001.xml.gz',
 '/user/eileen/test/test/pubmed21n0002.xml.gz',
 '/user/eileen/test/test/pubmed21n0003.xml.gz',
 '/user/eileen/test/test/pubmed21n0004.xml.gz',
 '/user/eileen/test/test/pubmed21n0005.xml.gz',
 '/user/eileen/test/test/pubmed21n0006.xml.gz',
 '/user/eileen/test/test/pubmed21n0007.xml.gz',
 '/user/eileen/test/test/pubmed21n0008.xml.gz',
 '/user/eileen/test/test/pubmed21n0009.xml.gz']

In [4]:
sc = spark.sparkContext

In [9]:
def parse_gzip_medline_str(gzip_str):
    import pubmed_parser as pp
    import unidecode
    

    filepath = gzip_str[0]
    xml_string = gzip_str[1]
    _, file_name = path.split(filepath)
    # decompress gzip

    articles = pp.parse_medline_xml(xml_string,nlm_category=True)
    return [Row(file_name=file_name, **article_dict)
            for article_dict in articles]

In [10]:
docs = sc.wholeTextFiles('test/test/pubmed21n0001.xml.gz').repartition(1000)
dicts = docs.flatMap(parse_gzip_medline_str)
df = dicts.toDF().toPandas()

In [23]:
df.dtypes


abstract         object
affiliation      object
author           object
country          object
delete             bool
file_name        object
issn_linking     object
journal          object
keywords         object
medline_ta       object
mesh_terms       object
nlm_unique_id    object
other_id         object
pmc              object
pmid             object
pubdate          object
title            object
dtype: object

In [27]:
for col in df.columns:
    if (col != 'delete' and not "".join(df[col])):
        print(col)
        

journal


In [ ]:
rdd0 = sc.parallelize(list(range(10000))).repartition(1000)

In [ ]:
def get_hostnode(e):
    import sys
    import socket
    
    return str(sys.version_info), str(socket.gethostname())


In [ ]:
def check_version(e):
    import pyarrow as pa
    hdfs = pa.hdfs.connect()
    
    return str(pa.__version__), str(hdfs)

In [ ]:
rdd0.map(check_version).distinct().collect()

In [ ]:
hdfs = pa.hdfs.connect()

In [ ]:
print(hdfs)

In [ ]:
rdd0.map(get_hostnode).distinct().collect()

In [ ]:
get_hostnode(2)

In [ ]:
!/home/tozeng/anaconda3/bin/python --version

In [35]:
re.__version__

'2.2.1'

In [ ]:
test_p = spark.sparkContext.parallelize(file_paths)
test_p.collect()

In [38]:
def find_journal(gzip):
    import re
    l = re.findall('<Title>[\s\S]*?<\/Title>', gzip[1])

    return(l)

In [ ]:
def test_medline_parquet(gzip):
    import pubmed_parser as pa
    dicts = pa.parse_medline_xml(gzip[1])
    return [Row(**doc) for doc in dicts]

In [ ]:
docs = spark.sparkContext.wholeTextFiles('test/test/pubmed21n0001.xml.gz')
dicts = docs.flatMap(find_journal)
dicts.take(10)

In [ ]:

def parse_gzip_medline_str(gzip_str):
    import pubmed_parser as pp
    import unidecode
    
    filepath = gzip_str[0]
    xml_string = gzip_str[1]
    _, file_name = path.split(filepath)
    # decompress gzip

    articles = pp.parse_medline_xml(xml_string)
    return [Row(file_name=file_name, **article_dict)
            for article_dict in articles]



In [ ]:
docs = spark.sparkContext.wholeTextFiles('test/test/pubmed21n0001.xml.gz')
dicts = docs.flatMap(parse_gzip_medline_str)
dicts.take(10)



In [ ]:
!hdfs dfs 

In [ ]:
spark.stop()

In [ ]:
def test_medline_parquet(file_path):
    import subprocess
    (ret, out, err)= run_cmd(['hdfs', 'dfs', '-ls', 'test/test/'])
    
    return (out,err)

In [ ]:
def run_cmd(args_list):
        """
        run linux commands
        """
        # import subprocess
        print('Running system command: {0}'.format(' '.join(args_list)))
        proc = subprocess.Popen(args_list, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        s_output, s_err = proc.communicate()
        s_return =  proc.returncode
        return s_return, s_output, s_err 

In [ ]:
(ret, out, err)= run_cmd(['hdfs', 'dfs', '-ls', 'test/test/'])
out.decode('ascii').split('\n')

In [ ]:
err = test_p.map(test_medline_parquet).take(1)

In [ ]:
err[0][1].decode('ascii').split('\n')

In [ ]:
tet = spark.sparkContext.wholeTextFiles('test/test/*.xml.gz')

In [ ]:
tet.map(lambda x : x[0]).take(1)

In [ ]:
pa.hdfs.connect()

In [ ]:
spark.stop()